In [1]:
from hedge_stand import HedgeST,HedgeSTBoll
from datetime import date, datetime
from demeter import TokenInfo, PoolBaseInfo, Actuator, Strategy, Asset, AccountStatus, BuyAction, SellAction, RowData, \
    ChainType
import optunity
import optunity.metrics
from decimal import Decimal
import pandas as pd
import  matplotlib.pylab as plt
# import logging 
# from logging import handlers
from load_data import pool_id_1_eth_u_500,pool_id_1_wbtc_u_3000

from strategy_ploter import  plot_position_return_decomposition

import quantstats as qs
qs.extend_pandas()

%matplotlib inline

def backtest_boll(a, hedge_spread_split,hedge_spread_rate,alpha,boll_period_n):
    # global RUNNING_TIME
    # print(f"==================spread running time {RUNNING_TIME}==================")

    decimal_a = Decimal(a).quantize(Decimal('0.00'))
    decimal_hedge_spread_split = Decimal(hedge_spread_split).quantize(Decimal('0.0'))
    decimal_hedge_spread_rate = Decimal(hedge_spread_rate).quantize(Decimal('0.00'))

    ema_max_spread_rate = round(0.2,4)



    #收益计算基础参数
    # net_value_base = 'ETH'

    runner_instance = Actuator(PAIR_POOL)
    # runner_instance.enable_notify = False
    runner_instance.strategy = HedgeSTBoll(a=decimal_a,hedge_spread_split = decimal_hedge_spread_split,hedge_spread_rate=decimal_hedge_spread_rate,alpha=alpha,ema_max_spread_rate=ema_max_spread_rate,period_n=int(boll_period_n))
    runner_instance.set_assets([Asset(USDC, 1000000)])
    save_path = f"../demeter/data/ETH/{POOL_ID}"
    runner_instance.data_path = save_path
    runner_instance.load_data(ChainType.Ethereum.name,
                                POOL_ID,
                                DATE_START,
                               DATE_END)
    runner_instance.run(enable_notify=False)

    hedge_count = runner_instance.strategy.hedge_count
    outside_ema_count = runner_instance.strategy.outside_ema_count

    # df_status = pd.DataFrame(runner_instance.account_status_list)

    total_net_value = runner_instance.final_status.net_value
    
    final_total_usdc_value = round(total_net_value + runner_instance.strategy.e.df['total'].iloc[-1],3)
    
    final_price = runner_instance.final_status.price

    final_total_eth_value = round(final_total_usdc_value / final_price,3)


    notice = f"ema+alpha:{RUNNING_TIME} times, a:{decimal_a}, hedge_spread_split:{decimal_hedge_spread_split}, hedge_spread_rate:{decimal_hedge_spread_rate},alpa:{alpha}"
    result =f" result: outside_ema_count:{outside_ema_count},hedge count:{hedge_count} final_total_eth_value:{final_total_eth_value},final_total_usdc_value:{final_total_usdc_value}"  
    print(notice)
    print(result)
    # if SEND_NOTICE:
    #     send_notice('CEX_Notify',notice)

    # RUNNING_TIME +=1

    # if NET_VALUE_BASE == 'USDC':
    #     print(final_total_usdc_value)
    #     return float(final_total_usdc_value)
    #     # profit_rate_usdc = profit_usdc / runner_instance.strategy.init_total_usdc
    # else:
    #     print(float(final_total_usdc_value / final_price))
    #     return float(final_total_usdc_value / final_price)


    return runner_instance


def backtest_instance(a, hedge_spread_split,hedge_spread_rate,alpha):
    # global RUNNING_TIME
    # print(f"==================spread running time {RUNNING_TIME}==================")

    decimal_a = Decimal(a).quantize(Decimal('0.00'))
    decimal_hedge_spread_split = Decimal(hedge_spread_split).quantize(Decimal('0.0'))
    decimal_hedge_spread_rate = Decimal(hedge_spread_rate).quantize(Decimal('0.00'))

    alpha = round(alpha,4)
    # ema_max_spread_rate = round(ema_max_spread_rate,4)

    eth = TokenInfo(name="eth", decimal=18)
    usdc = TokenInfo(name="usdc", decimal=6)
    pool = PoolBaseInfo(usdc, eth, 0.05, usdc)

    #收益计算基础参数
    # net_value_base = 'ETH'

    runner_instance = Actuator(pool)
    # runner_instance.enable_notify = False
    runner_instance.strategy = HedgeST(decimal_a,decimal_hedge_spread_split,decimal_hedge_spread_rate,alpha)
    runner_instance.set_assets([Asset(usdc, 1000000)])
    save_path = f"../demeter/data/ETH/{pool_id_1_eth_u_500}"
    runner_instance.data_path = save_path
    runner_instance.load_data(ChainType.Ethereum.name,
                                pool_id_1_eth_u_500,
                                DATE_START,
                               DATE_END)
    runner_instance.run(enable_notify=False)

    hedge_count = runner_instance.strategy.hedge_count
    outside_ema_count = runner_instance.strategy.outside_ema_count

    # df_status = pd.DataFrame(runner_instance.account_status_list)

    total_net_value = runner_instance.final_status.net_value
    
    final_total_usdc_value = round(total_net_value + runner_instance.strategy.e.df['total'].iloc[-1],3)
    
    final_price = runner_instance.final_status.price

    final_total_eth_value = round(final_total_usdc_value / final_price,3)

    #处理df_status
    df_status = pd.DataFrame(runner_instance.account_status_list)
    df_merge = pd.merge(e.df, df_status, on='timestamp', how='inner')

    df_merge['total_value'] = df_merge['total'] + df_merge['net_value']
    df_merge['total_value_eth'] = df_merge['total_value'] / df_merge['price']

    df_merge.set_index('timestamp', inplace=True)




    notice = f"ema+alpha:{RUNNING_TIME} times, a:{decimal_a}, hedge_spread_split:{decimal_hedge_spread_split}, hedge_spread_rate:{decimal_hedge_spread_rate},alpa:{alpha}"
    result =f" result: outside_ema_count:{outside_ema_count},hedge count:{hedge_count} final_total_eth_value:{final_total_eth_value},final_total_usdc_value:{final_total_usdc_value}"  
    print(notice)
    print(result)
    # if SEND_NOTICE:
    #     send_notice('CEX_Notify',notice)

    # RUNNING_TIME +=1

    # if NET_VALUE_BASE == 'USDC':
    #     print(final_total_usdc_value)
    #     return float(final_total_usdc_value)
    #     # profit_rate_usdc = profit_usdc / runner_instance.strategy.init_total_usdc
    # else:
    #     print(float(final_total_usdc_value / final_price))
    #     return float(final_total_usdc_value / final_price)


    return runner_instance





In [2]:

def generate_dfmerge(runner_instance):
    e = runner_instance.strategy.e

    total_net_value = runner_instance.final_status.net_value
        
    final_total_usdc_value = total_net_value + runner_instance.strategy.e.df['total'].iloc[-1]


    final_price = runner_instance.final_status.price
    # if net_value_base == 'USDC':
    profit_usdc = round(final_total_usdc_value- runner_instance.strategy.init_total_usdc,3)
    profit_rate_usdc = round(profit_usdc / runner_instance.strategy.init_total_usdc,3)
    # else:
    profit_eth = round(final_total_usdc_value/final_price - runner_instance.strategy.init_total_symbol,3)
    profit_rate_eth = round(profit_eth / runner_instance.strategy.init_total_symbol,3)

    print(f"USDC profit: {profit_usdc}, profit_rate: {profit_rate_usdc}")

    print(f"ETH profit: {profit_eth}, profit_rate: {profit_rate_eth}")

    df_status = pd.DataFrame(runner_instance.account_status_list)
    df_merge = pd.merge(e.df, df_status, on='timestamp', how='inner')

    df_merge['total_value'] = df_merge['total'] + df_merge['net_value']

    df_merge.set_index('timestamp', inplace=True)

    df_merge['total_value_profit_rate'] = df_merge['total_value'] / runner_instance.strategy.init_total_usdc


    df_merge['total_value_eth'] = df_merge['total_value'] / df_merge['price']

    df_merge['net_value_eth'] = df_merge['net_value'] / df_merge['price']

    df_merge['total_value_eth_profit_rate'] = df_merge['total_value_eth'] / runner_instance.strategy.init_total_symbol

    df_merge_f = df_merge.apply(pd.to_numeric, downcast='float')
   
    df_merge_f['net_value_rate'] = df_merge_f['net_value'] / df_merge_f['net_value'].iloc[0]

    df_merge_f['price_rate'] = df_merge_f['price'] / df_merge_f['price'].iloc[0]
    return df_merge_f

In [3]:
# pd.options.plotting.backend = "plotly"

CHAIN_ID = 'ETH'
NET_VALUE_BASE = 'WBTC'
RUNNING_TIME = 1
SEND_NOTICE = False
str_date_start = '2022-9-1'
str_date_end = '2022-11-30'
POOL_ID = pool_id_1_wbtc_u_3000
#eth pool
# eth = TokenInfo(name='eth', decimal=18)
# USDC = TokenInfo(name="usdc", decimal=6)
# PAIR_POOL = PoolBaseInfo(USDC, eth, 0.05, USDC)

# wbtc pool
wbtc = TokenInfo(name='wbtc', decimal=8)
USDC = TokenInfo(name="usdc", decimal=6)
PAIR_POOL = PoolBaseInfo(wbtc, USDC, 0.3, USDC)

DATE_START = datetime.strptime(str_date_start, "%Y-%m-%d").date()

DATE_END = datetime.strptime(str_date_end, "%Y-%m-%d").date()

# A1 test

In [4]:
a = 1.15
hedge_spread_split = 2.7
hedge_spread_rate = 0.9
alpha = 0.037
ema_max_spread_rate=0.2
period_n = 19

instance=backtest_boll(a=a,hedge_spread_split=hedge_spread_split,hedge_spread_rate=hedge_spread_rate,alpha=alpha,boll_period_n=period_n)

df_merge = generate_dfmerge(instance)

# fig = df_merge[['price_rate','total_value_eth_profit_rate','total_value_profit_rate']].plot().get_figure()

# fig.savefig('./fig/test.pdf')

# plot = df_merge[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

# plot.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n}.png")

# plot.show()

2022-12-26 14:24:17,105 - INFO - start load files from 2022-09-01 to 2022-11-30...


init parameters: a:1.15, hedge_spread_split:2.7, hedge_spread_rate:0.90,alpa:0.037,period_n:19



2022-12-26 14:24:19,000 - INFO - load file complete, preparing...
2022-12-26 14:24:23,153 - INFO - data has benn prepared
2022-12-26 14:24:23,154 - INFO - init strategy...


add ema column with alpha 0.037
130976


2022-12-26 14:24:24,488 - INFO - start main loop...


hedge rebalance ETH -24.80777935392767014527344056 20124.73558706320389991576924 profit: 0E-49
after rebalance=> lp total symbol:49.61555870785534029054688112, future amount:24.80777935392767014527344056, status:AccountStatus(timestamp=None, base_balance=Decimal('498500.0000000000000000000001'), quote_balance=Decimal('24.84504692431414135730940466'), base_uncollected=Decimal('0'), quote_uncollected=Decimal('0'), base_in_position=Decimal('0'), quote_in_position=Decimal('0'), net_value=Decimal('998500.0000000000000000000000'), price=Decimal('20124.73558706320389991576924'))
prepare to add LP: rate:1.15 price:20124.73558706320389991576924 down:17499.77007570713382601371238 up:23143.44592512268448490313463 init symbol amount:59.62811261835393980921343405,init usdc amount:1200000.000000000011102230246
eth_value 0.09164208460049842032122299 usdc value 0


 76%|████████████████████████████████████████████████████████████████████████████████▎                        | 100187/130976 [15:35<07:18, 70.15it/s]

2022-11-09 14:40:00  => last hedge sell ETH,price: 17418.91882260936842959775224,c_amount:51.31723202202516888093038184,f_amount:24.80777935392767014527344056,ema:17497.727890580845, trade amoutn: 26.50945266809749873565694128, hedge count:1
2022-11-09 14:40:00 out of range, price:17418.91882260936842959775224,ema:17497.727890580845 symbol:51.31723202202516888093038184, usdc:0
hedge rebalance ETH -32.45048114450310700204336053 17418.91882260936842959775224 profit: -67125.30526472922933317854692
after rebalance=> lp total symbol:61.49761566066655682331971962, future amount:30.74880783033327841165985981, status:AccountStatus(timestamp=None, base_balance=Decimal('534948.1451746326011959507682'), quote_balance=Decimal('30.78686084158548786855186008'), base_uncollected=Decimal('0'), quote_uncollected=Decimal('0'), base_in_position=Decimal('0'), quote_in_position=Decimal('0'), net_value=Decimal('1071221.974977181357194802907'), price=Decimal('17418.91882260936842959775224'))
prepare to add L

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 130976/130976 [23:28<00:00, 92.98it/s]
2022-12-26 14:47:53,273 - INFO - main loop finished, start calculate evaluating indicator...
2022-12-26 14:47:53,478 - INFO - run evaluating indicator
2022-12-26 14:47:53,543 - INFO - back testing finish


ema+alpha:1 times, a:1.15, hedge_spread_split:2.7, hedge_spread_rate:0.90,alpa:0.037
 result: outside_ema_count:0,hedge count:1 final_total_eth_value:94.394,final_total_usdc_value:1621699.449
USDC profit: 421699.449, profit_rate: 0.351
ETH profit: 34.765, profit_rate: 0.583


In [5]:
# qs.plots.snapshot(df_merge['total_value_eth'])

In [6]:


filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge['total_value_eth'],benchmark=df_merge['net_value_eth'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')


# A2 test

In [7]:
a = 1.15
hedge_spread_split = 2.7
hedge_spread_rate = 0.9
alpha = 0.037
ema_max_spread_rate=0.2
period_n2 = 9

instance2=backtest_boll(a=a,hedge_spread_split=hedge_spread_split,hedge_spread_rate=hedge_spread_rate,alpha=alpha,boll_period_n=period_n2)

df_merge2 = generate_dfmerge(instance2)

# fig = df_merge[['price_rate','total_value_eth_profit_rate','total_value_profit_rate']].plot().get_figure()

# fig.savefig('./fig/test.pdf')

# plot2 = df_merge2[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

# plot2.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n2}.png")

# plot2.show()
filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n2}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge2['total_value_eth'],benchmark=df_merge2['net_value_eth'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')

2022-12-26 14:48:38,163 - INFO - start load files from 2022-09-01 to 2022-11-30...


init parameters: a:1.15, hedge_spread_split:2.7, hedge_spread_rate:0.90,alpa:0.037,period_n:9



2022-12-26 14:48:39,973 - INFO - load file complete, preparing...
2022-12-26 14:48:44,341 - INFO - data has benn prepared
2022-12-26 14:48:44,342 - INFO - init strategy...


add ema column with alpha 0.037
130976


2022-12-26 14:48:45,721 - INFO - start main loop...


...

2022-11-21 03:15:00 hedge lower sell ETH,c_amount:52.00325566661011433189135152,f_amount:22.49767288376206409933046596, upper:16539.69,middle:16217.35,lower,15895.01, trade price: 15865.73216855066591037549036,ema:15894.61608448083,trade amount: 8.251134946571214312329393847, hedge count:14


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 129691/130976 [24:28<00:26, 48.09it/s]

2022-11-30 02:25:00 hedge up buy ETH, c_amount:38.05037064506926946116564036,f_amount:14.24653793719084978700107211,upper:16396.72,middle:15965.96,lower:,15535.20, trade price:16992.26336213316807798421694,ema:16868.60465147823, trade amount:8.251134946571214312329393847, hedge count:15


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 130976/130976 [24:54<00:00, 87.66it/s]
2022-12-26 15:13:39,871 - INFO - main loop finished, start calculate evaluating indicator...
2022-12-26 15:13:40,982 - INFO - run evaluating indicator
2022-12-26 15:13:41,072 - INFO - back testing finish


ema+alpha:1 times, a:1.15, hedge_spread_split:2.7, hedge_spread_rate:0.90,alpa:0.037
 result: outside_ema_count:0,hedge count:15 final_total_eth_value:91.657,final_total_usdc_value:1574688.975
USDC profit: 374688.975, profit_rate: 0.312
ETH profit: 32.029, profit_rate: 0.537


In [8]:
qs.reports.html(df_merge2['total_value_eth'],benchmark=df_merge2['net_value_eth'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')

# a3 ema hedge

In [9]:
a = 1.20
hedge_spread_split = 2.5
hedge_spread_rate = 0.86
alpha3 = 0.037
ema_max_spread_rate2=0.2

instance3=backtest_instance(a,hedge_spread_split,hedge_spread_rate,alpha3)

df_merge3 = generate_dfmerge(instance3)

filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge['total_value'],benchmark=df_merge['price'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')

# plot3 = df_merge3[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

# plot3.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha3}.png")

# plot3.show()
filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha3}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge3['total_value_eth'],benchmark=df_merge3['net_value_eth'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')

2022-12-26 15:15:15,122 - INFO - start load files from 2022-09-01 to 2022-11-30...


init parameters: a:1.20, hedge_spread_split:2.5, hedge_spread_rate:0.86,alpa:0.037,period_n:1440



2022-12-26 15:15:16,990 - INFO - load file complete, preparing...
2022-12-26 15:15:22,650 - INFO - data has benn prepared
2022-12-26 15:15:22,651 - INFO - init strategy...


add ema column with alpha 0.037
131040


2022-12-26 15:15:24,098 - INFO - start main loop...


...

2022-11-23 23:17:00 hedge buy ETH, trade price:1179.975538788221626077842653,ema:1173.8675417382785, trade amount:110.7321821515849039496553313, hedge count:39


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 129659/131040 [26:02<00:24, 55.67it/s]

2022-11-30 00:52:00 hedge buy ETH, trade price:1275.565760202385316453503081,ema:1239.1540996537217, trade amount:110.7321821515849039496553313, hedge count:40


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 131040/131040 [26:29<00:00, 82.44it/s]
2022-12-26 15:41:53,669 - INFO - main loop finished, start calculate evaluating indicator...
2022-12-26 15:41:55,146 - INFO - run evaluating indicator
2022-12-26 15:41:55,227 - INFO - back testing finish


NameError: name 'e' is not defined